In [11]:
# !pip install focal-loss
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation, Input, GlobalAveragePooling2D
from sklearn.model_selection import train_test_split
import shutil
from tensorflow.keras import backend as K
from tensorflow.keras import layers, models, regularizers
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from tensorflow.keras.optimizers.legacy import Adam
from focal_loss import BinaryFocalLoss
from tensorflow.keras.losses import BinaryFocalCrossentropy

Wykorzystany zbiór: https://zenodo.org/records/6378474

In [4]:
# glowny folder
base_dir = '/Users/milenabiernacka/Desktop/studia/DS/semestr2/PD-magisterka/Mushroom_dataset/cnn/'
# polaczenie glownego folderu i podfolderow z edible i poisonous

edible_dir = os.path.join(base_dir, 'Edible')
poisonous_dir = os.path.join(base_dir, 'Poisonous')


train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
    )

validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Assuming base_dir is the path to the dataset directory
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=16,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=16,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=16,
    class_mode='binary',
    shuffle=False
)

Found 1399 images belonging to 2 classes.
Found 300 images belonging to 2 classes.
Found 301 images belonging to 2 classes.


In [3]:
model = Sequential([
    Input(shape=(150, 150, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Use linear activation for Hinge loss
])


In [7]:
model.compile(optimizer=Adam(learning_rate=1e-4), loss=BinaryFocalLoss(gamma=5), metrics=['accuracy'])

In [8]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Epoch 1/20
87/87 [==============================] - 15s 162ms/step - loss: 0.0191 - accuracy: 0.7296 - val_loss: 0.0157 - val_accuracy: 0.7778
Epoch 2/20
87/87 [==============================] - 14s 160ms/step - loss: 0.0150 - accuracy: 0.7773 - val_loss: 0.0124 - val_accuracy: 0.8438
Epoch 3/20
87/87 [==============================] - 15s 170ms/step - loss: 0.0126 - accuracy: 0.8084 - val_loss: 0.0102 - val_accuracy: 0.8715
Epoch 4/20
87/87 [==============================] - 15s 168ms/step - loss: 0.0110 - accuracy: 0.8612 - val_loss: 0.0108 - val_accuracy: 0.8854
Epoch 5/20
87/87 [==============================] - 15s 177ms/step - loss: 0.0094 - accuracy: 0.8821 - val_loss: 0.0073 - val_accuracy: 0.8854
Epoch 6/20
87/87 [==============================] - 16s 184ms/step - loss: 0.0090 - accuracy: 0.8807 - val_loss: 0.0072 - val_accuracy: 0.9236
Epoch 7/20
87/87 [==============================] - 16s 182ms/step - loss: 0.0075 - accuracy: 0.9017 - val_loss: 0.0099 - val_accuracy: 0.8750

In [9]:
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

18/18 [==============================] - 3s 150ms/step - loss: 0.0032 - accuracy: 0.9618
Test Loss: 0.003221520222723484, Test Accuracy: 0.9618055820465088


In [3]:
train_generator.classes

array([0, 0, 0, ..., 1, 1, 1], dtype=int32)

In [4]:
train_generator.class_indices

{'Edible': 0, 'Poisonous': 1}

# Pretrenowane sieci

### MobileNetV2

In [5]:
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

validation_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Assuming base_dir is the path to the dataset directory
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(128, 128),
    batch_size=1,
    class_mode='binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

Found 1399 images belonging to 2 classes.
Found 300 images belonging to 2 classes.
Found 301 images belonging to 2 classes.


In [1]:
from tensorflow.keras.applications import MobileNetV2

In [2]:
img_height = img_width = 128

# Ładowanie MobileNetV2 bez górnej warstwy (top)
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Zamrożenie wagi przetrenowanego modelu
base_model.trainable = False

In [7]:
# Dodanie warstw na wierzchu
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
# To jest tylko przykład, można tutaj dostosować liczbę neuronów i warstw
output = Dense(1, activation='sigmoid')(x)  # Przykładowa warstwa wyjściowa dla klasyfikacji binarnej

In [9]:
# Skompletowanie nowego modelu
model = Model(inputs=base_model.input, outputs=output)

In [12]:
# Kompilacja modelu z niestandardową funkcją straty i optymalizatorem
model.compile(optimizer=Adam(lr=0.0001), loss=BinaryFocalCrossentropy(), metrics=['accuracy'])

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/optimizers/legacy/adam.py:118: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [13]:
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=50,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

Epoch 1/50
1399/1399 [==============================] - 13s 8ms/step - loss: 0.0616 - accuracy: 0.9299 - val_loss: 0.0303 - val_accuracy: 0.9549
Epoch 2/50
1399/1399 [==============================] - 13s 9ms/step - loss: 0.0245 - accuracy: 0.9693 - val_loss: 0.0136 - val_accuracy: 0.9757
Epoch 3/50
1399/1399 [==============================] - 15s 11ms/step - loss: 0.0236 - accuracy: 0.9707 - val_loss: 0.0094 - val_accuracy: 0.9792
Epoch 4/50
1399/1399 [==============================] - 16s 11ms/step - loss: 0.0242 - accuracy: 0.9793 - val_loss: 0.0119 - val_accuracy: 0.9861
Epoch 5/50
1399/1399 [==============================] - 18s 13ms/step - loss: 0.0116 - accuracy: 0.9857 - val_loss: 0.0075 - val_accuracy: 0.9931
Epoch 6/50
1399/1399 [==============================] - 16s 11ms/step - loss: 0.0122 - accuracy: 0.9864 - val_loss: 0.0173 - val_accuracy: 0.9931
Epoch 7/50
1399/1399 [==============================] - 14s 10ms/step - loss: 0.0111 - accuracy: 0.9836 - val_loss: 0.0100 - v

In [14]:
test_loss, test_accuracy = model.evaluate(test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

9/9 [==============================] - 1s 115ms/step - loss: 0.0211 - accuracy: 0.9861
Test Loss: 0.021085485816001892, Test Accuracy: 0.9861111044883728
